In [ ]:
from data_io import load_accelerometer_data
from feature_pipeline import AdaptiveAccelPipeline
from timeseries_models import train_lstm
from sklearn.preprocessing import LabelEncoder, StandardScaler

DATA_PATH = "/home/rajesh/work/acclerometer_project/zip_data"

df_raw = load_accelerometer_data(DATA_PATH)




{'Accuracy': 0.8601193705914271, 'F1': 0.8515045994185483}

In [ ]:
pipe = AdaptiveAccelPipeline(df_raw)
pipe.apply_adaptive_filter()
pipe.compute_physics_features()

df_win = pipe.resample_and_label(interval_seconds=20, coherence_threshold=0.7)

features = [c for c in df_win.columns if c not in ["subject", "local_ts", "label"]]

le = LabelEncoder()
df_win["label_enc"] = le.fit_transform(df_win["label"])

scaler = StandardScaler()
df_win[features] = scaler.fit_transform(df_win[features])

X, y = pipe.make_sequences(df_win, features, "label_enc", time_steps=5)

results = train_lstm(X, y, n_classes=len(le.classes_))
results

In [ ]:
results_table = []
    
    # Grid Search Settings
intervals = [10, 30]  # Window sizes in seconds
thresholds = [0.6, 0.8] # Strictness of labeling (60% vs 80% purity)
lstm_steps = [5, 10]    # How far back the LSTM looks

print(f"\n--- Starting Grid Search ---")
for iv in intervals:
    for th in thresholds:
        # 3. Resample and Label
        # This applies the logic: "Only keep window if > X% of data matches"
        df_ready = pipe.resample_and_label(interval_seconds=iv, coherence_threshold=th)
        
        if len(df_ready) < 100:
            print(f"Skipping Interval={iv}, Threshold={th} (Not enough valid windows)")
            continue

        # 4. Run Time Series Model
        ts_modeler = TimeSeriesModeler(pipeline, df_ready)
        
        for steps in lstm_steps:
            res = ts_modeler.run(time_steps=steps)
            if res:
                res['Window_Size_Sec'] = iv
                res['Label_Threshold'] = th
                results_table.append(res)

    # C. SAVE RESULTS
    if results_table:
        final_df = pd.DataFrame(results_table)
        final_df = final_df.sort_values(by='F1_Score', ascending=False)
        
        print("\n=== FINAL RESULTS ===")
        print(final_df)